In [221]:
import requests
import json
import pandas as pd

# locations is a dictionary of location names and coordinates
# coordinates in order [longitude, latitude]
# sources and dests are lists of location names
# gets driving distances
def get_locations_and_distances(locations: dict, source_names: list, dest_names: list, *, key: str = None, server: str='https://api.openrouteservice.org/v2/matrix/driving-car'):
    # remove duplicates
    source_names= list(set(source_names))
    dest_names= list(set(dest_names))
    
    # list of location coordinates to be passed into helper
    loc_coordinates= []
    # iterator initialized at zero which will help build the index lists to be passed into helper
    # make list of source indices
    source_index= []
    for i, s in enumerate(source_names):
        temp= locations.get(s)
        # if the location is not in the locations dictionary, raise exception
        if temp == None:
            raise Exception(f"Source site '{s}' not found in locations") 
        # add the coordinate matching the source 's'
        loc_coordinates.append(temp)
        source_index.append(i)
    source_count = len(source_names)
        
    # make list of dest indices
    dest_index= []
    for i, d in enumerate(dest_names):
        temp= locations.get(d)
        # if the location is not in the locations dictionary, raise exception
        if temp == None:
            raise Exception(f"Destination site '{d}' not found in locations")
            
        # add the coordinate matching the destination 'd'
        loc_coordinates.append(temp)
        dest_index.append(i+source_count)
    
    # call the helper to get the distances as a list
    return (source_names, dest_names, get_distances_helper(loc_coordinates, source_index, dest_index, server, key))
    
def convert_to_df(source_names, dest_names, distances):
    # convert list of distances to pandas dataframe
    values= []
    for source, dist in zip(source_names, distances):
        these_values= {'source': source}
        for dest, d in zip(dest_names, dist):
            these_values[dest]= d
        values.append(these_values)
        
    df= pd.DataFrame(values)
    return df

# locations should be the coordinates of each location in the list
# sources and destinations should be lists of the indexes of the chosen locations in locations
# key is your authorization key
# the returned list of distances is in kilometers
def get_distances_helper(locations: list, sources: list, dests: list, server: str, key: str) -> list:
    
    # initialize the body
    body = {"locations":locations, "destinations":dests, "metrics":["distance"], "sources":sources}
    
    # initialize the headers
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Content-Type': 'application/json; charset=utf-8'
    }

    if key:
        headers['Authorization']= key

    # exits if there is an error
    # may want to revise this block to include different responses to specific errors
    try:
        call = requests.post(server, json=body, headers=headers)
    except requests.exceptions.RequestException as e:
        print(call.reason)
    
    # get the json of the call
    jason = json.loads(call.text)
    return jason['distances']

def get_distances(locations: list, source_names: list, dest_names: list, *, server: str='https://api.openrouteservice.org/v2/matrix/driving-car', key: str=None, dataframe=pd.DataFrame()):
    if dataframe.empty:
        distances= get_locations_and_distances(locations, source_names, dest_names, key=key, server=server)
        return convert_to_df(distances[0], distances[1], distances[2]) 
    else:
        display(dataframe)
        # get list of sources and destinations that are already in current dataframe
        df_dests= dataframe.columns
        df_sources= list(dataframe['source'])
    
        # use those to find all sources and destinations in the list that aren't in the current dataframe
        new_sources= list(set(source_names).difference(set(df_sources)))
        new_dests= list(set(dest_names).difference(set(df_dests)))
    
        # We need to keep track of the source and destination of each distance
        if new_dests != []:
            
            new_columns= get_locations_and_distances(locations, df_sources, new_dests, key=key, server=server)
            # add the columns
            # for each new destination
            for dest, dists in zip(new_columns[1], new_columns[2]):
                # Define a dictionary with key values of
                # an existing column and their respective
                # value pairs as the # values for our new column.
                dests = {}
                for source, d in zip(new_columns[0], dists):
                    dests[source]= d

                df2= pd.DataFrame([dests])
                # Provide 'Address' as the column name
                dataframe= pd.merge(dataframe, df2, how="outer", on=["source"])
        if new_sources != []:
            new_rows= get_locations_and_distances(locations, new_sources, dest_names, key=key, server=server)
            
            # add the rows
            # for each new source
            for source, dists in zip(new_rows[0], new_rows[2]):
                new_values= {'source': source}
                for dest, d in zip(new_rows[1], dists):
                    new_values[dest]= d
                df2 = pd.DataFrame([new_values])
                # dataframe.append(new_values, ignore_index=True)
                dataframe= pd.concat([dataframe, df2], ignore_index=True)

            return dataframe

In [221]:
import requests
import json
import pandas as pd

# locations is a dictionary of location names and coordinates
# coordinates in order [longitude, latitude]
# sources and dests are lists of location names
# gets driving distances
def get_locations_and_distances(locations: dict, source_names: list, dest_names: list, *, key: str = None, server: str='https://api.openrouteservice.org/v2/matrix/driving-car'):
    # remove duplicates
    source_names= list(set(source_names))
    dest_names= list(set(dest_names))
    
    # list of location coordinates to be passed into helper
    loc_coordinates= []
    # iterator initialized at zero which will help build the index lists to be passed into helper
    # make list of source indices
    source_index= []
    for i, s in enumerate(source_names):
        temp= locations.get(s)
        # if the location is not in the locations dictionary, raise exception
        if temp == None:
            raise Exception(f"Source site '{s}' not found in locations") 
        # add the coordinate matching the source 's'
        loc_coordinates.append(temp)
        source_index.append(i)
    source_count = len(source_names)
        
    # make list of dest indices
    dest_index= []
    for i, d in enumerate(dest_names):
        temp= locations.get(d)
        # if the location is not in the locations dictionary, raise exception
        if temp == None:
            raise Exception(f"Destination site '{d}' not found in locations")
            
        # add the coordinate matching the destination 'd'
        loc_coordinates.append(temp)
        dest_index.append(i+source_count)
    
    # call the helper to get the distances as a list
    return (source_names, dest_names, get_distances_helper(loc_coordinates, source_index, dest_index, server, key))
    
def convert_to_df(source_names, dest_names, distances):
    # convert list of distances to pandas dataframe
    values= []
    for source, dist in zip(source_names, distances):
        these_values= {'source': source}
        for dest, d in zip(dest_names, dist):
            these_values[dest]= d
        values.append(these_values)
        
    df= pd.DataFrame(values)
    return df

# locations should be the coordinates of each location in the list
# sources and destinations should be lists of the indexes of the chosen locations in locations
# key is your authorization key
# the returned list of distances is in kilometers
def get_distances_helper(locations: list, sources: list, dests: list, server: str, key: str) -> list:
    
    # initialize the body
    body = {"locations":locations, "destinations":dests, "metrics":["distance"], "sources":sources}
    
    # initialize the headers
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Content-Type': 'application/json; charset=utf-8'
    }

    if key:
        headers['Authorization']= key

    # exits if there is an error
    # may want to revise this block to include different responses to specific errors
    try:
        call = requests.post(server, json=body, headers=headers)
    except requests.exceptions.RequestException as e:
        print(call.reason)
    
    # get the json of the call
    jason = json.loads(call.text)
    return jason['distances']

def get_distances(locations: list, source_names: list, dest_names: list, *, server: str='https://api.openrouteservice.org/v2/matrix/driving-car', key: str=None, dataframe=pd.DataFrame()):
    if dataframe.empty:
        distances= get_locations_and_distances(locations, source_names, dest_names, key=key, server=server)
        return convert_to_df(distances[0], distances[1], distances[2]) 
    else:
        display(dataframe)
        # get list of sources and destinations that are already in current dataframe
        df_dests= dataframe.columns
        df_sources= list(dataframe['source'])
    
        # use those to find all sources and destinations in the list that aren't in the current dataframe
        new_sources= list(set(source_names).difference(set(df_sources)))
        new_dests= list(set(dest_names).difference(set(df_dests)))
    
        # We need to keep track of the source and destination of each distance
        if new_dests != []:
            
            new_columns= get_locations_and_distances(locations, df_sources, new_dests, key=key, server=server)
            # add the columns
            # for each new destination
            for dest, dists in zip(new_columns[1], new_columns[2]):
                # Define a dictionary with key values of
                # an existing column and their respective
                # value pairs as the # values for our new column.
                dests = {}
                for source, d in zip(new_columns[0], dists):
                    dests[source]= d

                df2= pd.DataFrame([dests])
                # Provide 'Address' as the column name
                dataframe= pd.merge(dataframe, df2, how="outer", on=["source"])
        if new_sources != []:
            new_rows= get_locations_and_distances(locations, new_sources, dest_names, key=key, server=server)
            
            # add the rows
            # for each new source
            for source, dists in zip(new_rows[0], new_rows[2]):
                new_values= {'source': source}
                for dest, d in zip(new_rows[1], dists):
                    new_values[dest]= d
                df2 = pd.DataFrame([new_values])
                # dataframe.append(new_values, ignore_index=True)
                dataframe= pd.concat([dataframe, df2], ignore_index=True)

            return dataframe

In [21]:
get_distances_helper([[-77.0391047,38.8976763],[-76.6709638,39.1775715], [-76.4467045,39.0401319]], [2], [0, 1], 'https://api.openrouteservice.org/v2/matrix/driving-car', '5b3ce3597851110001cf624801643a3bcbdc4ebeaa0e59c7a33d9a8a') 

https://api.openrouteservice.org/v2/matrix/driving-car
{'locations': [[-77.0391047, 38.8976763], [-76.6709638, 39.1775715], [-76.4467045, 39.0401319]], 'destinations': [0, 1], 'metrics': ['distance'], 'sources': [2]}


[[58381.59, 34770.79]]

In [197]:
locations_dict= {'home': [-76.4467045, 39.0401319],
                  'white house': [-77.0391047,38.8976763],
                  'airport': [-76.6709638,39.1775715]}

api_key= '5b3ce3597851110001cf624801643a3bcbdc4ebeaa0e59c7a33d9a8a'
url='http://192.168.86.27:8080/ors/v2/matrix/driving-car'
source_names= ['home']
dest_names= ['white house', 'airport']



In [198]:
df1=get_distances(locations_dict, source_names, dest_names, server=url)

sources: ['home']
destinations: ['airport', 'white house']
locations: [[-76.4467045, 39.0401319], [-76.6709638, 39.1775715], [-77.0391047, 38.8976763]]
[{'source': 'home', 'airport': 34770.79, 'white house': 58381.51}]


In [224]:
# how to get destinations
df= get_distances(locations_dict, source_names, dest_names, server=url)
display(df)
columns= df.columns
# can treat columns like a list
print(columns)
print(len(columns))

sources: ['home', 'umd']
destinations: ['airport', 'white house']


,source,airport,white house
0,home,34770.79,58381.51
1,umd,47099.41,15976.50


Index(['source', 'airport', 'white house'], dtype='object')
3


In [100]:
# how to get sources
rows= list(df['source'])
print(len(rows))

1


In [226]:
locations_dict= {'home': [-76.4467045, 39.0401319],
                  'white house': [-77.0391047,38.8976763],
                  'airport': [-76.6709638,39.1775715], 'umd': [-76.9616087,38.9869183]}
source_names= ['home', 'umd']
dest_names= ['white house', 'airport']
get_distances(locations_dict, source_names, dest_names, server=url, dataframe=df)

,source,airport,white house
0,home,34770.79,58381.51
1,umd,47099.41,15976.50


In [162]:
# adding sources as destinations
dict= [{'source': 'umd', 'airport': 47099.41, 'white house': 15976.5}]
df1=pd.DataFrame(dict)

In [165]:
pd.concat([df2,df1], ignore_index=True)

,source,airport,white house
0,home,34770.79,58381.51
1,umd,47099.41,15976.50


In [189]:
# adding sources works for 1 source at least
df

,source,airport,white house
0,home,34770.79,58381.51


In [211]:
df2= get_distances(locations_dict, ['home','airport'], ['umd'], server=url)
df2

sources: ['home', 'airport']
destinations: ['umd']
locations: [[-76.4467045, 39.0401319], [-76.6709638, 39.1775715], [-76.9616087, 38.9869183]]
[{'source': 'home', 'umd': 52211.11}, {'source': 'airport', 'umd': 46898.64}]


,source,umd
0,home,52211.11
1,airport,46898.64


In [199]:
df1

,source,airport,white house
0,home,34770.79,58381.51


In [206]:
pd.concat([df1, df2], axis=1)

,source,airport,white house,source,umd
0,home,34770.79,58381.51,home,52211.11


In [215]:
pd.merge(df1, df2, how="outer", on=["source"])

,source,airport,white house,umd
0,home,34770.79,58381.51,52211.11
1,airport,NaN,NaN,46898.64
